In [ ]:
! pip3 install --upgrade pip
! pip install qiskit qiskit-nature
import qiskit
! pip install qiskit-aer
! pip install pyscf

In [ ]:
#VQE!!!
import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

qiskit_nature.settings.use_pauli_sum_op = False

from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2

In [ ]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)

    # Get properties
    properties = driver.run()

    # Reduce number of qubits used -> freeze core & remove 2 unoccupied orbitals
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    return qubit_op, num_particles, num_spatial_orbitals, problem, mapper


In [ ]:
from qiskit_aer.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP
import numpy as np
from qiskit.circuit import QuantumCircuit

#exact GSE is calculated on qubit operator and classical exact eigensolver.
#exact eigensolver (full config interaction (fci)): calculates exact eigenvalues and eigenvectors

def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result   # initial state |psi> is created -> VQE uses to produce final ansatz = min_theta (|psi(theta)>)

from qiskit_aer import Aer

def vqe_circuit_depth(qubit_op, num_particles, num_spatial_orbitals, qubit_converter):
    init_state = HartreeFock(num_spatial_orbitals, num_particles, qubit_converter)
    ansatz = UCCSD(num_spatial_orbitals, num_particles, qubit_converter, initial_state=init_state)
    circuit = QuantumCircuit(qubit_op.num_qubits)
    circuit.compose(init_state, inplace=True)
    circuit.compose(ansatz, inplace=True)
    return circuit.decompose().depth()

distances = np.arange(0.5, 4.0, 0.2)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=10)
noiseless_estimator = Estimator(approximation=True)

for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(
        dist
    )

    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies[0].real)
    vqe_depth = vqe_circuit_depth(qubit_op, num_particles, num_spatial_orbitals, qubit_converter)

    print(f"Interatomic Distance: {np.round(dist, 2)}")
    print(f"VQE Circuit Depth: {vqe_depth}")

print("All distances have been calculated")


Interatomic Distance: 0.5
VQE Circuit Depth: 26
Interatomic Distance: 0.7
VQE Circuit Depth: 26
Interatomic Distance: 0.9
VQE Circuit Depth: 26
Interatomic Distance: 1.1
VQE Circuit Depth: 26
Interatomic Distance: 1.3
VQE Circuit Depth: 26
Interatomic Distance: 1.5
VQE Circuit Depth: 26
Interatomic Distance: 1.7
VQE Circuit Depth: 26
Interatomic Distance: 1.9
VQE Circuit Depth: 26
Interatomic Distance: 2.1
VQE Circuit Depth: 26
Interatomic Distance: 2.3
VQE Circuit Depth: 26
Interatomic Distance: 2.5
VQE Circuit Depth: 26
Interatomic Distance: 2.7
VQE Circuit Depth: 26
Interatomic Distance: 2.9
VQE Circuit Depth: 26
Interatomic Distance: 3.1
VQE Circuit Depth: 26
Interatomic Distance: 3.3
VQE Circuit Depth: 26
Interatomic Distance: 3.5
VQE Circuit Depth: 26
Interatomic Distance: 3.7
VQE Circuit Depth: 26
Interatomic Distance: 3.9
VQE Circuit Depth: 26
All energies have been calculated
